In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize, LogNorm

from scipy.interpolate import RegularGridInterpolator
from tqdm import tqdm

Read **STEFAN HEIGLS** data. Data is currently (until 01.02.2022) available [here](https://gigamove.rwth-aachen.de/de/download/4abe80f1c550806021f85af8c57c886e).

In [ ]:
with np.load('../data/mhddisk.npz') as f:
    data = f['arr_0'][()]

# Visualization

## My own renderer

In [ ]:
import volrender
from astropy import constants as c
from matplotlib.colors import ListedColormap
from  matplotlib import cm

In [ ]:
vmax = data.max()
lognorm = LogNorm(vmin=vmax * 1e-4, vmax=vmax, clip=True)
datacube = lognorm(data.ravel()).reshape(data.shape).data

define a transfer function

In [ ]:
tf = volrender.TransferFunction(
    A  = [0.15, 0.2, 0.35],
    x0 = [0.1, 0.4, 0.92],
    sigma = [0.03, 0.05, 0.03],
    colors = np.array([
        [1.  , 0.5 , 0.  , 0.05],
        [0.25, 0.25, 0.75, 0.2 ],
        [1.  , 0.  , 0.25, 0.1 ]])
    )

create the renderer

In [ ]:
ren = volrender.Renderer(datacube, interactive=False, tf=tf)

In [ ]:
v = (c.pc/c.au).cgs.value

L  = 0.004 # box length in pc
L  *= v
M  = 1.19046233 # central mass in solar masses
# position of the sink
x0 = 0.46407932
y0 = 0.55431781
z0 = 0.49871549

In [ ]:
ren.render(theta=45, phi=128)

### Plotting

#### Single plot with colorbar

In [ ]:
ren.plot(norm=lognorm)

In [ ]:
f, ax = plt.subplots(dpi=200)
x = np.linspace(-L/2, L/2, data.shape[0] + 1)

data_sum = data.sum(2) * 0.004 * c.pc.to('cm').value
vmax = data_sum.max()
vmin = vmax*1e-3

cc=ax.pcolormesh(x, x, data_sum, norm=LogNorm(vmin=vmin, vmax=vmax), cmap='Blues')
ax.set_aspect('equal')
ax.set_xlabel('x [au]')
ax.set_ylabel('y [au]')

cb = plt.colorbar(cc, pad=0, ax=ax)
cb.set_label('column density [g/cm$^2$]', fontsize='small')

#### Plot with histogram

In [ ]:
f, ax = ren.plot(norm=lognorm, diagnostics=True)
f.subplots_adjust(wspace=0.5)

#### Make a movie

In [ ]:
N = 360
theta = np.linspace(45, 45, N)
phi = np.linspace(0, 359, N)

In [ ]:
fname = 'movie_mhddisk.mp4'
volrender.render_helper.render_movie(datacube, theta, phi, ncpu=10, fname=fname)

In [ ]:
from IPython.display import HTML
HTML(f"""
<video width="500" controls>
  <source src="{fname}" type="video/mp4">
</video>
""")

## Pyvista

In [ ]:
import pyvista

In [ ]:
grid = pyvista.UniformGrid()
grid.dimensions = np.array(data.shape) + 1
grid.spacing = (1, 1, 1)
grid.cell_data["density"] = np.log10(data.flatten(order="F"))

vmax = grid['density'].max()
grid['opac'] = Normalize(vmin=vmax - 1.5, vmax=vmax)(grid['density'])

In [ ]:
pyvista.close_all()

In [ ]:
p = pyvista.Plotter()
p.set_background('white')
p.add_volume(grid, scalars='opac', cmap="viridis", clim=[0, 1], opacity='linear', shade=False, opacity_unit_distance=.010)
p.show()